In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import urllib.parse
from sklearn import tree
from sklearn import metrics
import io
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
normal_file_raw = '/content/drive/MyDrive/csic 2010(rar)/normalTrafficTraining.txt'
anomaly_file_raw = '/content/drive/MyDrive/csic 2010(rar)/anomalousTrafficTest.txt'

normal_file_parse = 'normalRequestTraining.txt'
anomaly_file_parse = 'anomalousRequestTest.txt'

def parse_file(file_in, file_out):
    fin = open(file_in)
    fout = io.open(file_out, "w", encoding="utf-8")
    lines = fin.readlines()
    res = []
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith("GET"):
            res.append("GET" + line.split(" ")[1])
        elif line.startswith("POST") or line.startswith("PUT"):
            url = line.split(' ')[0] + line.split(' ')[1]
            j = 1
            while True:
                if lines[i + j].startswith("Content-Length"):
                    break
                j += 1
            j += 1
            data = lines[i + j + 1].strip()
            url += '?' + data
            res.append(url)
    for line in res:
        line = urllib.parse.unquote(line).replace('\n','').lower()
        fout.writelines(line + '\n')
    print ("finished parse ",len(res)," requests")
    fout.close()
    fin.close()



def loadData(file):
    with open(file, 'r', encoding="utf8") as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if (len(d) > 0):
            result.append(d)
    return result



parse_file(normal_file_raw,normal_file_parse)
parse_file(anomaly_file_raw,anomaly_file_parse)

bad_requests = loadData('anomalousRequestTest.txt')
good_requests = loadData('normalRequestTraining.txt')

all_requests = bad_requests + good_requests
yBad = [1] * len(bad_requests)
yGood = [0] * len(good_requests)
y = yBad + yGood
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3))
X = vectorizer.fit_transform(all_requests)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)

finished parse  36000  requests
finished parse  25065  requests


In [ ]:
#linear svm and decision tree(stacking)
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a toy dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create base classifiers
svm = LinearSVC(random_state=42)
tree = DecisionTreeClassifier(random_state=42)

# Create ensemble model using stacking
estimators = [('svm', svm), ('tree', tree)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())

# Train the ensemble model
stacking_model.fit(X_train, y_train)

# Evaluate the model on the testing set
score = stacking_model.score(X_test, y_test)
print(f"Accuracy: {score}")

Accuracy: 0.9945963648272474


In [ ]:
#random forest and linearsvm(stacking)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Load iris dataset
#iris = load_iris()

# Split dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train a linear SVM model
svm = LinearSVC(random_state=42)
svm.fit(X_train, y_train)

# Train a Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Ensemble by taking the majority vote of predicted classes
svm_pred = svm.predict(X_test)
rf_pred = rf.predict(X_test)
ensemble_pred = (svm_pred + rf_pred) / 2  # Take average of predicted probabilities
ensemble_pred = [int(round(x)) for x in ensemble_pred]  # Convert probabilities to class labels

# Calculate accuracy of ensemble model
accuracy = accuracy_score(y_test, ensemble_pred)
print(f"Ensemble model accuracy: {accuracy}")

Ensemble model accuracy: 0.9895202226952677


In [ ]:
#decision tree , linear svm and random forest(voting)
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# load the data and labels
#X = np.load('data.npy')
#y = np.load('labels.npy')

# split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize the models
svm_model = LinearSVC()
dt_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()

# initialize the ensemble model
ensemble_model = VotingClassifier(estimators=[('svm', svm_model), ('dt', dt_model), ('rf', rf_model)], voting='hard')

# train the ensemble model on the training set
ensemble_model.fit(X_train, y_train)

# evaluate the ensemble model on the testing set
y_pred = ensemble_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", acc)


Accuracy: 0.9967250695922711


In [ ]:
#decision tree and logistic regression(voting)
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a random binary classification dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the individual classifiers
svm_clf = svm.LinearSVC(random_state=42)
tree_clf = DecisionTreeClassifier(random_state=42)
logit_clf = LogisticRegression(random_state=42,max_iter=3000)

# Define the ensemble classifier with majority voting
ensemble_clf = VotingClassifier(estimators=[('svm', svm_clf), ('tree', tree_clf), ('logit', logit_clf)], voting='hard')

# Train the ensemble classifier
ensemble_clf.fit(X_train, y_train)

# Predict the test set labels using the ensemble classifier
y_pred = ensemble_clf.predict(X_test)

# Evaluate the accuracy of the ensemble classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the ensemble classifier: {:.2f}%".format(accuracy * 100))

Accuracy of the ensemble classifier: 99.71%


In [ ]:
#linear svm and RF(voting)
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a toy dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the individual classifiers
svm_clf = LinearSVC(random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the voting classifier
voting_clf = VotingClassifier(estimators=[('svm', svm_clf), ('rf', rf_clf)], voting='hard')

# Train the voting classifier
voting_clf.fit(X_train, y_train)

# Predict the test set labels using the voting classifier
y_pred = voting_clf.predict(X_test)

# Calculate the accuracy of the voting classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the voting classifier:", accuracy)


Accuracy of the voting classifier: 0.9895202226952677


In [ ]:
#linear svm and decision tree(blending)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a sample classification dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base models
model1 = LinearSVC(random_state=42)
model2 = DecisionTreeClassifier(random_state=42)

# Fit the base models on the training data
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

# Make predictions on the test data
pred1 = model1.predict(X_test)
pred2 = model2.predict(X_test)

# Combine the predictions using blending
pred = (pred1 + pred2) / 2

# Evaluate the blended model
accuracy = (pred == y_test).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.9891927296544949


In [ ]:
##linear svm and decision tree(bagging)
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a random dataset for demonstration purposes
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base classifiers
svm = LinearSVC(random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# Define the ensemble classifier using bagging
ensemble = BaggingClassifier(base_estimator=[svm, dt], n_estimators=10, random_state=42)

# Train the ensemble classifier
ensemble.fit(X_train, y_train)

# Evaluate the performance of the ensemble classifier on the testing set
score = ensemble.score(X_test, y_test)
print(f"Ensemble accuracy score: {score:.2f}")


In [ ]:
#linearsvm and kneighbors(stacking)
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load your dataset and split it into X and y

# Define the base estimators
estimators = [
    ('svm', make_pipeline(StandardScaler(), LinearSVC())),
    ('knn', KNeighborsClassifier())
]

# Define the stacking estimator
stacking_estimator = StackingClassifier(
    estimators=estimators,
    final_estimator=LinearSVC()
)

# Use cross-validation to get the predicted labels for the base estimators
X_train_meta = []
for estimator in estimators:
    y_train_meta = cross_val_predict(estimator[1], X_train, y_train, cv=5, method='predict')
    X_train_meta.append(y_train_meta.reshape(-1, 1))

X_train_meta = np.hstack(X_train_meta)

# Train the stacking estimator on the predicted labels from the base estimators
stacking_estimator.fit(X_train_meta, y_train)

# Use the trained ensemble to make predictions on the test set
X_test_meta = []
for estimator in estimators:
    y_test_meta = estimator[1].predict(X_test)
    X_test_meta.append(y_test_meta.reshape(-1, 1))

X_test_meta = np.hstack(X_test_meta)

y_pred = stacking_estimator.predict(X_test_meta)

# Calculate the accuracy score of the ensemble predictions
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
#linearsvm and kneighbors (voting)
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a random dataset for classification
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual classifiers
svm_clf = LinearSVC(random_state=42)
knn_clf = KNeighborsClassifier()

# Create the ensemble classifier
ensemble_clf = VotingClassifier(estimators=[('svm', svm_clf), ('knn', knn_clf)], voting='hard')

# Train the ensemble classifier
ensemble_clf.fit(X_train, y_train)

# Evaluate the ensemble classifier
y_pred = ensemble_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9828066153594236


In [ ]:
#decision tree and logistic regression(voting)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a toy dataset for demonstration
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create individual classifiers
decision_tree = DecisionTreeClassifier(random_state=42)
logistic_regression = LogisticRegression(random_state=42)

# Create an ensemble using VotingClassifier
ensemble = VotingClassifier(
    estimators=[('dt', decision_tree), ('lr', logistic_regression)],
    voting='soft'  # Use 'hard' voting for majority rule, 'soft' for probability averaging
)

# Fit the ensemble on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = ensemble.predict(X_test)

# Calculate accuracy of the ensemble
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9909939413787457


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#decision tree and logistic reg. using stacking
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification

# Generate a synthetic dataset for demonstration
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize base models
base_models = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('logistic_regression', LogisticRegression(random_state=42))
]

# Initialize the stacking classifier with the base models
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Fit the stacking model on the training data
stacking_model.fit(X_train, y_train)

# Evaluate the stacking model on the testing data
accuracy = stacking_model.score(X_test, y_test)
print(f"Stacking Model Accuracy: {accuracy:.2f}")
print("Accuracy of the ensemble classifier: {:.2f}%".format(accuracy * 100))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Stacking Model Accuracy: 0.99
Accuracy of the ensemble classifier: 98.98%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#decision tree and logistic regression(blending)
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Generate some synthetic data for classification
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classifiers
tree_clf = DecisionTreeClassifier(random_state=42)
logreg_clf = LogisticRegression(random_state=42)

# Train the classifiers
tree_clf.fit(X_train, y_train)
logreg_clf.fit(X_train, y_train)

# Make predictions on the test set
tree_preds = tree_clf.predict_proba(X_test)[:, 1]
logreg_preds = logreg_clf.predict_proba(X_test)[:, 1]

# Define blending weights (can be tuned)
tree_weight = 0.6
logreg_weight = 0.4

# Combine predictions using blending
ensemble_preds = tree_weight * tree_preds + logreg_weight * logreg_preds

# Round predictions to obtain binary labels
ensemble_preds_binary = np.round(ensemble_preds)

# Evaluate the ensemble's performance
accuracy = np.mean(ensemble_preds_binary == y_test)
print("Ensemble Accuracy:", accuracy)


Ensemble Accuracy: 0.9909939413787457


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#logistic regression and decision tree(bagging)
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a synthetic dataset
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base classifiers
tree_classifier = DecisionTreeClassifier()
logistic_classifier = LogisticRegression()

# Create an ensemble using bagging
bagging_tree = BaggingClassifier(base_estimator=tree_classifier, n_estimators=10, random_state=42)
bagging_logistic = BaggingClassifier(base_estimator=logistic_classifier, n_estimators=10, random_state=42)

# Fit the ensemble models on the training set
bagging_tree.fit(X_train, y_train)
bagging_logistic.fit(X_train, y_train)

# Make predictions on the testing set
tree_predictions = bagging_tree.predict(X_test)
logistic_predictions = bagging_logistic.predict(X_test)

# Calculate ensemble accuracies
tree_accuracy = accuracy_score(y_test, tree_predictions)
logistic_accuracy = accuracy_score(y_test, logistic_predictions)

print("Ensemble using Decision Trees Accuracy:", tree_accuracy)
print("Ensemble using Logistic Regression Accuracy:", logistic_accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs fail

Ensemble using Decision Trees Accuracy: 0.9901752087768135
Ensemble using Logistic Regression Accuracy: 0.9800229245128541


In [ ]:
#decision tree and logistic regression(boosting)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a synthetic dataset for classification
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize decision tree and logistic regression classifiers
tree_clf = DecisionTreeClassifier(max_depth=3)
logreg_clf = LogisticRegression()

# Initialize the AdaBoost classifier with decision tree as base estimator
boosted_clf = AdaBoostClassifier(base_estimator=tree_clf, n_estimators=50)

# Fit the ensemble model on the training data
boosted_clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = boosted_clf.predict(X_test)

# Calculate and print the accuracy of the ensemble model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.9656132307188472


In [ ]:
#kneighbors , randomforest(blending)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier(n_neighbors=5)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
knn.fit(X_train, y_train)
rf.fit(X_train, y_train)
knn_preds = knn.predict(X_test)
rf_preds = rf.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

print("KNN Accuracy:", knn_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
ensemble_preds = (knn_preds + rf_preds) / 2
ensemble_accuracy = accuracy_score(y_test, ensemble_preds)
print("Ensemble Accuracy:", ensemble_accuracy)


In [ ]:
#randomforest , logisticregression(voting)
sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

rf_clf = RandomForestClassifier(random_state=42)
lr_clf = LogisticRegression(random_state=42)

voting_clf = VotingClassifier(estimators=[('rf', rf_clf), ('lr', lr_clf)], voting='hard')

# Fitting the Voting Classifier to the training data
voting_clf.fit(X_train, y_train)

# Evaluating the Voting Classifier on the test data
score = voting_clf.score(X_test, y_test)

# Printing the accuracy score of the Voting Classifier
print("Voting Classifier Accuracy:", score)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Voting Classifier Accuracy: 0.9811691501555592


In [ ]:
#randomforest and logisticregression(voting)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
rf_classifier = RandomForestClassifier(random_state=42)
lr_classifier = LogisticRegression(random_state=42)

# Create the ensemble using voting
ensemble = VotingClassifier(estimators=[('rf', rf_classifier), ('lr', lr_classifier)], voting='hard')

# Fit the ensemble on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the test data
y_pred = ensemble.predict(X_test)

# Calculate the accuracy of the ensemble
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9811691501555592


In [ ]:
#randomforest , logisticregression(blending)
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
rf_preds = rf_model.predict_proba(X_test)[:, 1]  
lr_preds = lr_model.predict_proba(X_test)[:, 1]  
rf_weight = 0.5
lr_weight = 0.5

blended_preds = rf_weight * rf_preds + lr_weight * lr_preds

ensemble_preds = np.round(blended_preds)
accuracy = np.mean(ensemble_preds == y_test)
print("Ensemble Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.9923039135418372


In [ ]:
#randomforest , logisticregression(bagging)
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
random_forest = RandomForestClassifier()
logistic_regression = LogisticRegression()

# Create ensemble using BaggingClassifier
ensemble = BaggingClassifier(base_estimator=[random_forest, logistic_regression])

# Train the ensemble model
ensemble.fit(X, y)

# Make predictions using the ensemble model
predictions = ensemble.predict(X)

# Evaluate the ensemble model
accuracy = ensemble.score(X, y)
print("Accuracy:", accuracy)


InvalidParameterError: ignored

In [ ]:
#randomforest , logisticregression (bagging)
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
lr_classifier = LogisticRegression(random_state=42)
ensemble_classifier = BaggingClassifier(base_estimator=[rf_classifier, lr_classifier], n_estimators=10, random_state=42)
ensemble_classifier.fit(X_train, y_train)
y_pred = ensemble_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Accuracy:", accuracy)


InvalidParameterError: ignored

In [ ]:
#randomforest , randomforest(bagging)
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate some sample data
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create base classifiers
logistic_regression = LogisticRegression(random_state=42)
random_forest = RandomForestClassifier(random_state=42)

# Create an ensemble using BaggingClassifier
ensemble = BaggingClassifier(base_estimator=logistic_regression,
                             n_estimators=10,
                             random_state=42)

# Train the ensemble
ensemble.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = ensemble.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

Ensemble Accuracy: 0.9800229245128541


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#randomforest , logisticregression(boosting)
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
logistic_model = LogisticRegression()
rf_model = RandomForestClassifier()
boosted_model = AdaBoostClassifier(base_estimator=rf_model, n_estimators=50)
boosted_model.fit(X_train, y_train)
y_pred = boosted_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.985917799246766


In [ ]:
#decisiontree and decisiontree(stacking)
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
classifier1 = LogisticRegression(random_state=42)
classifier2 = DecisionTreeClassifier(random_state=42)
estimators = [('lr', classifier1), ('dt', classifier2)]
stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_classifier.fit(X_train, y_train)
y_pred = stacking_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.9898477157360406


In [ ]:
#logisticregression , decisiontree(voting)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset and split it into training and testing sets
#X, y = load_data()  # Replace with your own data loading code
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize individual models
logistic_model = LogisticRegression()
tree_model = DecisionTreeClassifier()

# Train individual models
logistic_model.fit(X_train, y_train)
tree_model.fit(X_train, y_train)

# Make predictions on the testing set
logistic_preds = logistic_model.predict(X_test)
tree_preds = tree_model.predict(X_test)

# Create a meta-model (e.g., logistic regression) and use the predictions from individual models as inputs
blending_model = LogisticRegression()
blending_model.fit(np.column_stack((logistic_preds, tree_preds)), y_test)

# Make predictions using the blending model
blending_preds = blending_model.predict(np.column_stack((logistic_preds, tree_preds)))

# Evaluate the performance of the blending model
accuracy = accuracy_score(y_test, blending_preds)
print("Blending Model Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Blending Model Accuracy: 0.9909939413787457


In [ ]:
#logisticregression decisiontree(boosting)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
dt = DecisionTreeClassifier(max_depth=1)
boost_lr = AdaBoostClassifier(base_estimator=lr, n_estimators=50, random_state=42)
boost_dt = AdaBoostClassifier(base_estimator=dt, n_estimators=50, random_state=42)
boost_lr.fit(X_train, y_train)
boost_dt.fit(X_train, y_train)
y_pred_lr = boost_lr.predict(X_test)
y_pred_dt = boost_dt.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print("Boosting with Logistic Regression Accuracy:", accuracy_lr)
print("Boosting with Decision Tree Accuracy:", accuracy_dt)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Boosting with Logistic Regression Accuracy: 0.7000163746520387
Boosting with Decision Tree Accuracy: 0.9277877845095792


In [ ]:
#logisticregression,decisiontree (boosting)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
tree_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=1,
    random_state=42
)
tree_model.fit(X_train, y_train)

# Evaluate the models
lr_pred = lr_model.predict(X_test)
tree_pred = tree_model.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))
print("Decision Tree Ensemble Accuracy:", accuracy_score(y_test, tree_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.9816603897167185
Decision Tree Ensemble Accuracy: 0.9608645816276404


In [ ]:
#logisticregression and decisiontree(bagging)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
lr = LogisticRegression()
dt = DecisionTreeClassifier()
bag_lr = BaggingClassifier(base_estimator=lr, n_estimators=10, random_state=42)
bag_dt = BaggingClassifier(base_estimator=dt, n_estimators=10, random_state=42)
bag_lr.fit(X_train, y_train)
bag_dt.fit(X_train, y_train)
print("Bagging Logistic Regression score:", bag_lr.score(X_test, y_test))
print("Bagging Decision Tree score:", bag_dt.score(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

Bagging Logistic Regression score: 0.9800229245128541
Bagging Decision Tree score: 0.9901752087768135


In [ ]:
#logisticregression and linearsvm(stacking)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
logreg_model = LogisticRegression()
svm_model = LinearSVC()
estimators = [('logreg', logreg_model), ('svm', svm_model)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)
y_pred = stacking_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 99.75%


In [ ]:
#logisticregression , linearsvm()
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
logistic_clf = LogisticRegression()
svm_clf = LinearSVC()

logistic_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
logistic_preds = logistic_clf.predict(X_test)
svm_preds = svm_clf.predict(X_test)
ensemble_preds = (logistic_preds + svm_preds) / 2
ensemble_preds = ensemble_preds.round()
accuracy = accuracy_score(y_test, ensemble_preds)
print("Ensemble accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble accuracy: 0.9875552644506305


In [ ]:
#logisticregression , linearsvm(blending)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
logistic_model = LogisticRegression()
svm_model = LinearSVC()
logistic_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Make predictions on the test data
logistic_preds = logistic_model.predict(X_test)
svm_preds = svm_model.predict(X_test)

# Blend the predictions using a simple averaging scheme
blended_preds = (logistic_preds + svm_preds) / 2

# Round the blended predictions to the nearest integer
blended_preds = blended_preds.round().astype(int)

# Evaluate the accuracy of the ensemble predictions
accuracy = accuracy_score(y_test, blended_preds)
print("Ensemble Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.9875552644506305


In [ ]:
#logisticregression , linearsvm(bagging)
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
svm = LinearSVC()

# Create the ensemble using bagging
ensemble = BaggingClassifier(base_estimator=logreg, n_estimators=10)

# Train the ensemble
ensemble.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ensemble.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

Accuracy: 0.9806779105943999


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#logisticregression,linearsvm(voting)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
logistic_regression = LogisticRegression()
linear_svm = LinearSVC()
voting_classifier = VotingClassifier(
    estimators=[('lr', logistic_regression), ('svm', linear_svm)],
    voting='hard'
)
voting_classifier.fit(X_train, y_train)
y_pred = voting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9875552644506305


In [ ]:
#decisiontree , linearsvm and logisticregression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
tree_model = DecisionTreeClassifier(random_state=42)
logreg_model = LogisticRegression(random_state=42)
svm_model = LinearSVC(random_state=42)

ensemble_model = VotingClassifier(
    estimators=[('tree', tree_model), ('logreg', logreg_model), ('svm', svm_model)],
    voting='hard'  # Use 'hard' voting, where the majority vote wins
)
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ensemble_model.predict(X_test)

# Evaluate the ensemble model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.9970525626330441


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.9970525626330441


In [ ]:
#decisiontree, logisticregression and linearsvm(stacking)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
tree_clf = DecisionTreeClassifier(random_state=42)
log_reg_clf = LogisticRegression(random_state=42)
svm_clf = LinearSVC(random_state=42)
estimators = [
    ('tree', tree_clf),
    ('log_reg', log_reg_clf),
    ('svm', svm_clf)
]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = stacking_clf.predict(X_test)

# Calculate the accuracy of the ensemble classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.9978712952349763


In [ ]:
#decision tree ,logisticregression and linearsvm(bagging)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset and split it into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the base classifiers
decision_tree = DecisionTreeClassifier()
logistic_regression = LogisticRegression()
linear_svm = LinearSVC()

# Initialize the ensemble classifier using bagging
ensemble = BaggingClassifier(base_estimator=None, n_estimators=10, random_state=42)

# Train the ensemble classifier
ensemble.fit(X_train, y_train)

# Make predictions using the ensemble classifier
y_pred = ensemble.predict(X_test)

# Calculate the accuracy of the ensemble classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9901752087768135


In [ ]:
#decisiontree , logisticregression and linearsvm(boosting)
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Generate some synthetic data
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize individual classifiers
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
lr = LogisticRegression(random_state=42)
svm = LinearSVC(random_state=42)

# Initialize the boosting ensemble
ensemble = AdaBoostClassifier( random_state=42)

# Add individual classifiers to the ensemble
ensemble.estimators_ = [dt, lr, svm]

# Fit the ensemble on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the test data
y_pred = ensemble.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Ensemble Accuracy: {accuracy}")


Ensemble Accuracy: 0.9277877845095792


In [ ]:
#decision tree ,logistic regression and linearsvm(blending)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your data and split it into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define individual classifiers
decision_tree = DecisionTreeClassifier()
logistic_regression = LogisticRegression()
linear_svc = LinearSVC()

# Train individual classifiers
decision_tree.fit(X_train, y_train)
logistic_regression.fit(X_train, y_train)
linear_svc.fit(X_train, y_train)

# Make predictions on the test set using individual classifiers
dt_predictions = decision_tree.predict(X_test)
lr_predictions = logistic_regression.predict(X_test)
svc_predictions = linear_svc.predict(X_test)

# Create an ensemble by blending the predictions of individual classifiers
ensemble_predictions = []
for dt_pred, lr_pred, svc_pred in zip(dt_predictions, lr_predictions, svc_predictions):
    # Voting logic: Majority voting
    if dt_pred + lr_pred + svc_pred >= 2:
        ensemble_predictions.append(1)
    else:
        ensemble_predictions.append(0)

# Calculate the accuracy of the ensemble predictions
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.9975438021942034


In [2]:
#randomforest , logisticregression and linearsvm(stacking)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

# Load your dataset and split it into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create base models
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
logistic_regression = LogisticRegression(random_state=42)
linear_svm = LinearSVC(random_state=42)

# Create the stacking ensemble
estimators = [
    ('random_forest', random_forest),
    ('logistic_regression', logistic_regression),
    ('linear_svm', linear_svm)
]

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Train the stacking ensemble
stacking_classifier.fit(X_train, y_train)

# Make predictions
predictions = stacking_classifier.predict(X_test)

# Evaluate the ensemble
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.9981987882757491


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a synthetic dataset for demonstration
#X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the classifiers
rf_clf = RandomForestClassifier(random_state=42)
lr_clf = LogisticRegression(random_state=42)
svm_clf = LinearSVC(random_state=42)

# Create the voting classifier
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('lr', lr_clf), ('svm', svm_clf)],
    voting='hard'  # Use 'soft' for probabilities-based voting
)

# Train the voting classifier
voting_clf.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
